In [21]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [22]:
train = pd.read_csv('C:/222/large_data/kaggle/mnist/train.csv')
test = pd.read_csv('C:/222/large_data/kaggle/mnist/test.csv')
path = 'C:/222/large_data/kaggle/mnist/'

In [23]:
validate = train.iloc[0:100]
train = train.iloc[100:len(train)]
train_y = train['label']
validate_y = validate['label']
train = train.drop(columns = ['label'])
validate = validate.drop(columns = ['label'])

In [24]:
batch_size = 16
classes = train_y.unique()
num_classes = np.shape(classes)[0]

In [25]:
def labels_onehot(vector, num_classes = num_classes):
    y_true_enc = np.zeros((len(vector), num_classes))
    for i in range(len(vector)):
        y_true_enc[i][vector.iloc[i]] = 1
    return y_true_enc

In [26]:
train_labels = labels_onehot(vector = train_y)
validate_labels = labels_onehot(vector = validate_y)

In [27]:
def rows2array(data):
    data = np.array(data).reshape(-1, 28, 28)
    return np.array(data).reshape(-1, 28, 28, 1)

In [28]:
train_data = rows2array(train)
validate_data = rows2array(validate)

In [29]:
test_data = rows2array(test)

In [39]:
training_iters = 20
learning_rate = 1e-3 
batch_size = int(train_data.shape[0]/16)

In [31]:
#both placeholders are of type float
x = tf.placeholder("float", shape = [None, train_data.shape[1], train_data.shape[2], train_data.shape[3]], name = 'x')
y = tf.placeholder("float", shape = [None, num_classes], name = 'y')

In [32]:
def conv2d(x, W_kernel, bias, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W_kernel, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, bias)
    return tf.nn.relu(x) 

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [33]:
weights = {
    'wc1': tf.get_variable('W0', shape=(3,3,1,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc2': tf.get_variable('W1', shape=(3,3,32,64), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc3': tf.get_variable('W2', shape=(3,3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'wd1': tf.get_variable('W3', shape=(4*4*128,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('W6', shape=(128, num_classes), initializer=tf.contrib.layers.xavier_initializer()), 
}
biases = {
    'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B4', shape=(10), initializer=tf.contrib.layers.xavier_initializer()),
}

ValueError: Variable W0 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Anaconda2\lib\site-packages\tensorflow\python\framework\ops.py", line 1801, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Anaconda2\lib\site-packages\tensorflow\python\framework\ops.py", line 3300, in create_op
    op_def=op_def)
  File "C:\Anaconda2\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)


In [ ]:
def conv_net(x, weights, biases,  name = 'prediction'):  

    # here we call the conv2d function we had defined above and pass the input image x, weights wc1 and bias bc1.
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 14*14 matrix.
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    # here we call the conv2d function we had defined above and pass the input image x, weights wc2 and bias bc2.
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 7*7 matrix.
    conv2 = maxpool2d(conv2, k=2)

    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 4*4.
    conv3 = maxpool2d(conv3, k=2)


    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'], name = 'pred')
    return out

In [34]:
pred = conv_net(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [35]:
#Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

#calculate accuracy across all the given images and average them out. 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [36]:
init = tf.global_variables_initializer()

In [41]:
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init) 
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
#     summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    for i in range(training_iters):
        for batch in range(int(len(train_data)/batch_size)):
            print(batch)
            batch_x = train_data[batch*batch_size:min((batch+1)*batch_size,len(train_data))]
            batch_y = train_labels[batch*batch_size:min((batch+1)*batch_size, len(train_labels))]
#             print('y batch sh:', batch_y.shape)
#             print('x batch sh:', batch_x.shape)
            # Run optimization op (backprop).
                # Calculate batch loss and accuracy
            opt = sess.run(optimizer, feed_dict={x: batch_x, 
                                                 y: 
                                                 batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y})
        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
        print("Optimization Finished!")

        test_acc, valid_loss = sess.run([accuracy,cost], feed_dict={x: validate_data, y : validate_labels})
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Training accuracy:", "{:.5f}".format(acc))
        print("Testing Accuracy:","{:.5f}".format(test_acc))
        test_f_ans = sess.run(tf.argmax(pred, 1), feed_dict = {x: test_data})
        print(len(test_f_ans))
        if i >= 15:
            np.savetxt(path + 'answers_' + str(i) + 'txt', test_f_ans.astype(int))
    save_path = saver.save(sess, "C:/222/large_data/kaggle/mnist/my_model")
#     summary_writer.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Iter 0, Loss= 2.057037, Training Accuracy= 0.41940
Optimization Finished!
Training accuracy: 0.41940
Testing Accuracy: 0.46000
28000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Iter 1, Loss= 0.703264, Training Accuracy= 0.82468
Optimization Finished!
Training accuracy: 0.82468
Testing Accuracy: 0.85000
28000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Iter 2, Loss= 0.226120, Training Accuracy= 0.93354
Optimization Finished!
Training accuracy: 0.93354
Testing Accuracy: 0.92000
28000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Iter 3, Loss= 0.138274, Training Accuracy= 0.95493
Optimization Finished!
Training accuracy: 0.95493
Testing Accuracy: 0.92000
28000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Iter 4, Loss= 0.096647, Training Accuracy= 0.96715
Optimization Finished!
Training accuracy: 0.96715
Testing Accuracy: 0.96000
28000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Iter 5, Loss= 0.075549, Training Accuracy= 0.97555
Optimization Finished!
Training accuracy: 0.97555
Testin

In [ ]:
# tf.reset_default_graph()

# # Create some variables.
# # v1 = tf.get_variable("v1", shape=[3])
# # v2 = tf.get_variable("v2", shape=[5])

# # Add ops to save and restore all the variables.
# # saver = tf.train.Saver()

# # Later, launch the model, use the saver to restore variables from disk, and
# # do some work with the model.
# with tf.Session() as sess:
    
#     # Restore variables from disk.
#     saver.restore(sess, "C:/222/large_data/kaggle/mnist.ckpt")
#     print("Model restored.")
#     # Check the values of the variables
#     print("eval", test_data.eval())
# #     print("v2 : %s" % v2.eval())